### Notebook to do methylation calling with nanopolish

In [1]:
import os
from Bio import SeqIO

In [2]:
notebook_path = os.path.abspath(".")

In [3]:
#####One OUT_DIR per treatment. This should be one for germinated spores and one for infected leaves
IN_FAST5 = os.path.abspath('../../analyses/single_fast5s/infected_leaves/mapped_fast5s')
in_fastq_fn = os.path.abspath('../../data/genomic_data/infected_leaves/all_fastq/infected_leaves_all.all.fastq')
seq_sum_fn = os.path.abspath('../../data/genomic_data/infected_leaves/sequencing_summary/infected_leaves_all.sequencing_summary.txt')
#####One OUT_DIR per treatment. This should be one for germinated spores and one for infected leaves
OUT_DIR = os.path.abspath('../../analyses/pycometh/infected_leaves')
meth_freq_script = '/home/jamila/jamila_Storage/scripts/nanopolish_scripts/calculate_methylation_frequency.py'
meth_compare_script = '/home/jamila/jamila_Storage/scripts/nanopolish_scripts/co'

In [5]:
ref_genome = os.path.abspath('../../analyses/pycometh/chr_A_B_unassigned.fasta')
n_threads = 20

In [6]:
###file names for rerunning 
in_fastq_clean_fn = os.path.join(OUT_DIR, os.path.basename(in_fastq_fn).replace('all.fastq','clean_all.fastq'))
bam_fn = os.path.basename(ref_genome).replace('.fa', '') \
+'.'+ os.path.basename(in_fastq_clean_fn).replace('.fastq', '.bam')
bam_fn = os.path.join(OUT_DIR, bam_fn)
meth_call_fn = bam_fn.replace('.bam', '.meth_call.tsv')
meth_freq_fn = meth_call_fn.replace( '.meth_call.tsv', '.meth_freq.tsv')

# Section 1 checking the input

In [ ]:
#filer the fastq files to line up with the fast5 files
fast5s_names = set([x.replace('.fast5','') for x in os.listdir(IN_FAST5)])

In [ ]:
fastqs = []
for seq in SeqIO.parse(in_fastq_fn, 'fastq'):
    if seq.id in fast5s_names:
        fastqs.append(seq)

In [ ]:
in_fastq_clean_fn = os.path.join(OUT_DIR, os.path.basename(in_fastq_fn).replace('all.fastq','clean_all.fastq'))

In [ ]:
if len(fastqs) == len(fast5s_names):
    in_fastq_clean_fn = os.path.join(OUT_DIR, os.path.basename(in_fastq_fn).replace('all.fastq','clean_all.fastq'))
    with open(in_fastq_clean_fn,'w') as fh:
        SeqIO.write(fastqs, fh, 'fastq')
    print('fastq subsetted')
    !head {in_fastq_clean_fn}

# Section 2 nanopolish methylation calling

In [ ]:
#change directory
os.chdir(OUT_DIR)
basename = os.path.basename(OUT_DIR)

In [ ]:
%%capture cap_out_index
%time
!nanopolish index -d {IN_FAST5} {in_fastq_clean_fn}

In [ ]:
###print stdout
print(cap_out_index.stdout)

In [ ]:
###print stderr
print(cap_out_index.stderr)

In [ ]:
bam_fn = os.path.basename(ref_genome).replace('.fa', '') \
+'.'+ os.path.basename(in_fastq_clean_fn).replace('.fastq', '.bam')
bam_fn = os.path.join(OUT_DIR, bam_fn)
print(bam_fn)

In [ ]:
#%%capture cap_out_map
%time
!minimap2 -a -t 20 -x map-ont {ref_genome} {in_fastq_clean_fn} | samtools sort -@20 -T tmp -o {bam_fn}
!samtools index -@20 {bam_fn}

In [ ]:
%%capture cap_out_methcall
%time
!nanopolish call-methylation -t 20 -r {in_fastq_clean_fn} -b {bam_fn} -g {ref_genome} > {meth_call_fn}

In [ ]:
print(cap_out_methcall.stdout)

In [ ]:
!python3 {meth_freq_script} {meth_call_fn} > {meth_freq_fn}

In [ ]:
!head -200 {meth_freq_fn}

## Comparative methylation calling with pycometh

In [7]:
os.chdir(notebook_path)

In [8]:
PYCO_OUT_DIR = os.path.abspath('../../pycometh/comparative/')

In [9]:
from pycoMeth.CpG_Aggregate import CpG_Aggregate
from pycoMeth.Interval_Aggregate import Interval_Aggregate
from pycoMeth.CGI_Finder import CGI_Finder
from pycoMeth.Meth_Comp import Meth_Comp
# Optionally inport jupyter helper functions
from pycoMeth.common import head, jhelp
import sys

In [10]:
##setup
min_depth = 3
sample_id = 'infected_leaves'

In [11]:
cpg_agg_bed_fn = meth_call_fn.replace('.meth_call', '.CpG_agg').replace('.tsv','.bed')
cpg_agg_tsv_fn = meth_call_fn.replace('.meth_call', '.CpG_agg').replace('.tsv','.tsv.gz')

In [12]:
ff = CpG_Aggregate(nanopolish_fn= meth_call_fn,
    ref_fasta_fn=ref_genome,
    output_bed_fn=cpg_agg_bed_fn,
    output_tsv_fn=cpg_agg_tsv_fn,
    sample_id=sample_id,
    progress=True)

## Checking options and input files ##
## Parsing methylation_calls file ##
	Starting to parse file Nanopolish methylation call file
	Progress: 100%|██████████| 3.14G/3.14G [03:50<00:00, 13.7M bytes/s]
	Parsing summary
		Lines Parsed: 30,825,533
		Line successfully parsed: 30,825,533
		Input files: 1
	Filtering out low coverage sites
	Sorting each chromosome by coordinates
	Sites summary
		Total Valid Lines: 30,825,533
		Initial Sites: 3,270,496
		Low Count Sites: 1,828,922
		Valid Sites Found: 1,441,574
## Processing valid sites found and write to file ##
	Progress: 100%|██████████| 1.44M/1.44M [02:59<00:00, 8.01k sites/s]
	Results summary
		Total Sites Writen: 1,441,574
		Unmethylated sites: 657,050
		Ambiguous sites: 435,359
		Methylated sites: 349,165


In [13]:
for fn in [500, 1000, 5000]:
    interval = fn
    int_agg_bed_fn = meth_call_fn.replace('.meth_call', '.interval_%s_agg' % interval).replace('.tsv','.bed')
    int_agg_tsv_fn = meth_call_fn.replace('.meth_call', '.interval_%s_agg' % interval).replace('.tsv','.tsv.gz')
    #print(int_agg_bed_fn, '\n', int_agg_tsv_fn)
    fg = Interval_Aggregate(
    cpg_aggregate_fn=cpg_agg_tsv_fn,
    ref_fasta_fn=ref_genome,
    output_bed_fn=int_agg_bed_fn,
    output_tsv_fn=int_agg_tsv_fn,
    interval_size=interval,
    min_cpg_per_interval=1,
    sample_id=sample_id,
    progress=True)

## Checking options and input files ##
## Parsing CpG_aggregate file ##
	Progress: 172M bytes [00:31, 5.45M bytes/s]                        
	Results summary
		Lines parsed: 1,441,574
		Total number of intervals: 353,009
	Writter summary
		Empty intervals skipped: 194,000
		Valid intervals written: 159,009
## Checking options and input files ##
## Parsing CpG_aggregate file ##
	Progress: 172M bytes [00:26, 6.45M bytes/s]                        
	Results summary
		Lines parsed: 1,441,574
		Total number of intervals: 176,557
	Writter summary
		Valid intervals written: 89,540
		Empty intervals skipped: 87,017
## Checking options and input files ##
## Parsing CpG_aggregate file ##
	Progress: 172M bytes [00:17, 9.78M bytes/s]                        
	Results summary
		Lines parsed: 1,441,574
		Total number of intervals: 35,389
	Writter summary
		Valid intervals written: 24,910
		Empty intervals skipped: 10,479


In [14]:
###CGI island finder -> error might fix it
fl = CGI_Finder (
    ref_fasta_fn=ref_genome,
    output_bed_fn=ref_genome.replace(".fasta", ".CGI.bed"),
    output_tsv_fn=ref_genome.replace(".fasta", ".CGI.tsv"),
    progress=True)

## Checking options and input files ##
## Parsing reference fasta file ##
	Parsing Reference sequence: chr1A
	Progress: 100%|██████████| 6.16M/6.16M [00:05<00:00, 1.07M bases/s]
	Parsing Reference sequence: chr2A
	Progress: 100%|██████████| 6.06M/6.06M [00:05<00:00, 1.09M bases/s]
	Parsing Reference sequence: chr3A
	Progress: 100%|██████████| 6.03M/6.03M [00:05<00:00, 1.04M bases/s]
	Parsing Reference sequence: chr4A
	Progress: 100%|██████████| 5.97M/5.97M [00:05<00:00, 1.05M bases/s]
	Parsing Reference sequence: chr5A
	Progress: 100%|██████████| 5.56M/5.56M [00:05<00:00, 1.03M bases/s]
	Parsing Reference sequence: chr6A
	Progress: 100%|██████████| 5.55M/5.55M [00:05<00:00, 1.08M bases/s]
	Parsing Reference sequence: chr7A
	Progress: 100%|██████████| 5.18M/5.18M [00:04<00:00, 1.08M bases/s]
	Parsing Reference sequence: chr8A
	Progress: 100%|██████████| 5.11M/5.11M [00:04<00:00, 1.07M bases/s]
	Parsing Reference sequence: chr9A
	Progress: 100%|██████████| 4.79M/4.79M [00:04<00:00, 1.07M

	Progress: 100%|██████████| 37.4k/37.4k [00:00<00:00, 1.13M bases/s]
	Parsing Reference sequence: tig00000329
	Progress: 100%|██████████| 103k/103k [00:00<00:00, 1.02M bases/s]
	Parsing Reference sequence: tig00000331
	Progress: 100%|██████████| 51.5k/51.5k [00:00<00:00, 1.01M bases/s]
	Parsing Reference sequence: tig00000332
	Progress: 100%|██████████| 105k/105k [00:00<00:00, 1.04M bases/s]
	Parsing Reference sequence: tig00000333
	Progress: 100%|██████████| 24.0k/24.0k [00:00<00:00, 986k bases/s]
	Parsing Reference sequence: tig00000486
	Progress: 100%|██████████| 29.0k/29.0k [00:00<00:00, 1.01M bases/s]
	Parsing Reference sequence: tig00000487
	Progress: 100%|██████████| 63.2k/63.2k [00:00<00:00, 1.02M bases/s]
	Parsing Reference sequence: tig00000489
	Progress: 100%|██████████| 57.2k/57.2k [00:00<00:00, 1.03M bases/s]
	Parsing Reference sequence: tig00000491
	Progress: 100%|██████████| 25.0k/25.0k [00:00<00:00, 980k bases/s]
	Parsing Reference sequence: tig00000493
	Progress: 100%|

	Progress: 100%|██████████| 32.4k/32.4k [00:00<00:00, 1.04M bases/s]
	Parsing Reference sequence: tig00001174
	Progress: 100%|██████████| 88.9k/88.9k [00:00<00:00, 937k bases/s]
	Parsing Reference sequence: tig00001177
	Progress: 100%|██████████| 46.2k/46.2k [00:00<00:00, 1.08M bases/s]
	Parsing Reference sequence: tig00001179
	Progress: 100%|██████████| 95.2k/95.2k [00:00<00:00, 968k bases/s]
	Parsing Reference sequence: tig00001182
	Progress: 100%|██████████| 21.7k/21.7k [00:00<00:00, 973k bases/s]
	Parsing Reference sequence: tig00001189
	Progress: 100%|██████████| 29.5k/29.5k [00:00<00:00, 1.13M bases/s]
	Parsing Reference sequence: tig00001197
	Progress: 100%|██████████| 28.8k/28.8k [00:00<00:00, 994k bases/s]
	Parsing Reference sequence: tig00001199
	Progress: 100%|██████████| 17.9k/17.9k [00:00<00:00, 1.04M bases/s]
	Parsing Reference sequence: tig00001202
	Progress: 100%|██████████| 33.3k/33.3k [00:00<00:00, 1.14M bases/s]
	Parsing Reference sequence: tig00001206
	Progress: 100

		Number of reference sequences: 207


In [20]:
ref_genome.replace(".fasta", ".CGI.bed")

'/home/jamila/jamila_Storage/analyses/pycometh/chr_A_B_unassigned.CGI.bed'

In [21]:
ref_genome.replace(".fasta", ".CGI.tsv")

'/home/jamila/jamila_Storage/analyses/pycometh/chr_A_B_unassigned.CGI.tsv'

In [15]:
cpg_agg_il_tsv = cpg_agg_tsv_fn
cpg_agg_sp_tsv = '/home/jamila/jamila_Storage/analyses/pycometh/germinated_spores/chr_A_B_unassignedsta.germinated_spores_1.clean_all.CpG_agg.tsv.gz'

In [16]:
ref_genome

'/home/jamila/jamila_Storage/analyses/pycometh/chr_A_B_unassigned.fasta'

In [17]:
fg = Meth_Comp (
    aggregate_fn_list=[cpg_agg_il_tsv, cpg_agg_sp_tsv],
    ref_fasta_fn=ref_genome,
    output_bed_fn=os.path.join(PYCO_OUT_DIR, 'Pgt21-0.comp_CpG.il_sp.bed'),
    output_tsv_fn=os.path.join(PYCO_OUT_DIR, 'Pgt21-0.comp_CpG.il_sp.tsv.gz'),
    sample_id_list=['infected_leaves',"spores"],
    max_missing = 1,
    min_diff_llr = 0,
    progress=True)

## Checking options and input files ##
## Parsing files ##
	Reading input files header and checking consistancy between headers
	Starting asynchronous file parsing
	Progress: 322M bytes [00:36, 8.72M bytes/s]                       
	Adjust pvalues
	Writing output file
	Progress: 100%|██████████| 34.2k/34.2k [00:02<00:00, 15.4k sites/s]
	Results summary
		Sites with insufficient samples: 1,551,785
		Sites with insufficient effect size: 553,811
		Valid sites: 34,242
		Sites with non-significant adjusted pvalue: 33,945
		Sites with non-significant pvalue: 31,623
		Sites with significant pvalue: 2,619
		Sites with significant adjusted pvalue: 297


In [18]:
int_agg_il_tsv = "/home/jamila/jamila_Storage/analyses/pycometh/infected_leaves/chr_A_B_unassignedsta.infected_leaves_all.clean_all.interval_1000_agg.tsv.gz"
int_agg_sp_tsv = "/home/jamila/jamila_Storage/analyses/pycometh/germinated_spores/chr_A_B_unassignedsta.germinated_spores_1.clean_all.interval_1000_agg.tsv.gz"
fg = Meth_Comp (
    aggregate_fn_list=[cpg_agg_il_tsv, cpg_agg_sp_tsv],
    ref_fasta_fn=ref_genome,
    output_bed_fn=os.path.join(PYCO_OUT_DIR, 'Pgt21-0.comp_w1000.il_sp.bed'),
    output_tsv_fn=os.path.join(PYCO_OUT_DIR, 'Pgt21-0.comp_w1000.il_sp.tsv.gz'),
    sample_id_list=['infected_leaves',"spores"],
    max_missing = 1,
    min_diff_llr = 1,
    progress=True)

## Checking options and input files ##
## Parsing files ##
	Reading input files header and checking consistancy between headers
	Starting asynchronous file parsing
	Progress: 322M bytes [00:28, 11.2M bytes/s]                       
	Adjust pvalues
	Writing output file
	Progress: 100%|██████████| 6.17k/6.17k [00:00<00:00, 19.0k sites/s]
	Results summary
		Sites with insufficient samples: 1,551,785
		Sites with insufficient effect size: 581,884
		Valid sites: 6,169
		Sites with non-significant adjusted pvalue: 6,169
		Sites with non-significant pvalue: 5,468
		Sites with significant pvalue: 701


In [19]:
int_agg_il_tsv = "/home/jamila/jamila_Storage/analyses/pycometh/infected_leaves/chr_A_B_unassignedsta.infected_leaves_all.clean_all.interval_500_agg.tsv.gz"
int_agg_sp_tsv = "/home/jamila/jamila_Storage/analyses/pycometh/germinated_spores/chr_A_B_unassignedsta.germinated_spores_1.clean_all.interval_500_agg.tsv.gz"
fg = Meth_Comp (
    aggregate_fn_list=[cpg_agg_il_tsv, cpg_agg_sp_tsv],
    ref_fasta_fn=ref_genome,
    output_bed_fn=os.path.join(PYCO_OUT_DIR, 'Pgt21-0.comp_w500.il_sp.bed'),
    output_tsv_fn=os.path.join(PYCO_OUT_DIR, 'Pgt21-0.comp_w500.il_sp.tsv.gz'),
    sample_id_list=['infected_leaves',"spores"],
    max_missing = 1,
    min_diff_llr = 1,
    progress=True)

## Checking options and input files ##
## Parsing files ##
	Reading input files header and checking consistancy between headers
	Starting asynchronous file parsing
	Progress: 322M bytes [00:28, 11.5M bytes/s]                       
	Adjust pvalues
	Writing output file
	Progress: 100%|██████████| 6.17k/6.17k [00:00<00:00, 19.6k sites/s]
	Results summary
		Sites with insufficient samples: 1,551,785
		Sites with insufficient effect size: 581,884
		Valid sites: 6,169
		Sites with non-significant adjusted pvalue: 6,169
		Sites with non-significant pvalue: 5,468
		Sites with significant pvalue: 701
